## roster model estimation

To determine the impact each roster position has on team success, we need to examine the quality of players per game and the result of each game. For each roster position, there will be elite players and secondary players per team. Elite players will be assinged a value of 1 whereas secondary players, a value 2.

### import data sets  "play by play goal detail" and "game detail"

In [48]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pylab import hist, show
import scipy
import zipfile


pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)


In [49]:
pwd

'/Users/stefanostselios/Desktop/nhl_roster_design-master'

In [50]:
d0 = pd.read_csv('season_games.csv', index_col=0)
d1 = pd.read_csv('game_team_roster_quality.csv', index_col=0)

In [58]:
d0['WinTeam'] = d0.apply(lambda x: 'HOME' if x['GD'] > 0 else 'AWAY', axis=1)
d0 = d0[['Season', 'GameNumber', 'VTeamCode', 'HTeamCode', 'HGF', 'VGF', 'GD', 'WinTeam']]
d0.head()

,Season,GameNumber,VTeamCode,HTeamCode,HGF,VGF,GD,WinTeam
0,2010,20001,MTL,TOR,3,2,1,HOME
1,2010,20002,PHI,PIT,2,3,-1,AWAY
2,2010,20003,CAR,MIN,3,4,-1,AWAY
3,2010,20004,CHI,COL,4,3,1,HOME
4,2010,20005,CGY,EDM,4,0,4,HOME


In [59]:
dm = d0.merge(d1, on=['Season', 'GameNumber'], how='left')

- Calculate the difference between player quality per game for all positions with respect to home team ( Home Team - Visitor Team). There are 5 positions and 2 types of player quality. This will give us a total of 10 differenecs. 

In [62]:
dm = dm[dm['GameNumber'] <= 21230]
dm.shape

(1230, 16)

In [63]:
dm['VF'] = dm['VF1'] + dm['VF2']
dm['VD'] = dm['VD1'] + dm['VD2']
dm['HF'] = dm['HF1'] + dm['HF2']
dm['HD'] = dm['HD1'] + dm['HD2']

In [64]:
dm['VF'].value_counts()

12.0    1114
11.0      75
13.0      31
10.0       9
Name: VF, dtype: int64

In [65]:
dm['VD'].value_counts()

6.0    1114
7.0      75
5.0      31
8.0       9
Name: VD, dtype: int64

In [66]:
dm['HF'].value_counts()

12.0    1122
11.0      91
13.0       3
Name: HF, dtype: int64

In [67]:
dm['HD'].value_counts()

6.0    1122
7.0      91
5.0       3
Name: HD, dtype: int64

In [68]:
dm = dm[((dm['VF'] == 12) & (dm['HF'] == 12) & (dm['VD'] == 6) & (dm['HD'] == 6))]

In [69]:
dm['VD'].value_counts()

6.0    1015
Name: VD, dtype: int64

In [70]:
dm['HD'].value_counts()

6.0    1015
Name: HD, dtype: int64

In [71]:
dm['VF'].value_counts()

12.0    1015
Name: VF, dtype: int64

In [72]:
dm['HF'].value_counts()

12.0    1015
Name: HF, dtype: int64

## Summary analysis

In [73]:
dm.describe()

,Season,GameNumber,HGF,VGF,GD,VF1,VF2,VD1,VD2,HF1,HF2,HD1,HD2,VF,VD,HF,HD
count,1015.0,1015.000000,1015.000000,1015.000000,1015.000000,1015.000000,1015.000000,1015.000000,1015.000000,1015.000000,1015.000000,1015.000000,1015.000000,1015.0,1015.0,1015.0,1015.0
mean,2010.0,20622.621675,2.941872,2.739901,0.201970,2.805911,9.194089,1.121182,4.878818,3.209852,8.790148,1.266010,4.733990,12.0,6.0,12.0,6.0
std,0.0,352.180753,1.716485,1.634000,2.438513,1.519357,1.519357,0.685216,0.685216,1.583414,1.583414,0.667783,0.667783,0.0,0.0,0.0,0.0
min,2010.0,20001.000000,0.000000,0.000000,-8.000000,0.000000,6.000000,0.000000,4.000000,0.000000,6.000000,0.000000,4.000000,12.0,6.0,12.0,6.0
25%,2010.0,20319.500000,2.000000,2.000000,-1.000000,2.000000,8.000000,1.000000,4.000000,2.000000,7.000000,1.000000,4.000000,12.0,6.0,12.0,6.0
50%,2010.0,20628.000000,3.000000,3.000000,1.000000,3.000000,9.000000,1.000000,5.000000,3.000000,9.000000,1.000000,5.000000,12.0,6.0,12.0,6.0
75%,2010.0,20927.500000,4.000000,4.000000,2.000000,4.000000,10.000000,2.000000,5.000000,5.000000,10.000000,2.000000,5.000000,12.0,6.0,12.0,6.0
max,2010.0,21230.000000,9.000000,10.000000,7.000000,6.000000,12.000000,2.000000,6.000000,6.000000,12.000000,2.000000,6.000000,12.0,6.0,12.0,6.0


In [74]:
dm = dm[['Season', 'GameNumber', 'VTeamCode', 'HTeamCode', 'HGF', 'VGF', 'GD','WinTeam',
         'VF1', 'VF2', 'VD1', 'VD2', 
         'HF1', 'HF2', 'HD1', 'HD2']]

In [75]:
dm['HomeWin'] = dm.apply(lambda x: 1 if x['WinTeam']=='HOME' else 0, axis=1)
dm['DF1'] = dm['HF1'] - dm['VF1']
dm['DF2'] = dm['HF2'] - dm['VF2']
dm['DD1'] = dm['HD1'] - dm['VD1']
dm['DD2'] = dm['HD2'] - dm['VD2']

In [76]:
dm.groupby(['WinTeam'])['DF1', 'DF2', 'DD1', 'DD2'].describe()

DF1         DF2         DD1         DD2
WinTeam                                                      
AWAY    count  486.000000  486.000000  486.000000  486.000000
        mean     0.261317   -0.261317    0.080247   -0.080247
        std      2.204874    2.204874    0.967375    0.967375
        min     -5.000000   -6.000000   -2.000000   -2.000000
        25%     -1.000000   -2.000000   -1.000000   -1.000000
        50%      0.000000    0.000000    0.000000    0.000000
        75%      2.000000    1.000000    1.000000    1.000000
        max      6.000000    5.000000    2.000000    2.000000
HOME    count  529.000000  529.000000  529.000000  529.000000
        mean     0.534972   -0.534972    0.204159   -0.204159
        std      2.204336    2.204336    0.992348    0.992348
        min     -5.000000   -6.000000   -2.000000   -2.000000
        25%     -1.000000   -2.000000    0.000000   -1.000000
        50%      1.000000   -1.000000    0.000000    0.000000
        75%      2.000000    1.000000    1.000000    0.000000
        max      6.000000    5.000000    2.000000    2.000000

## Mean number of F1, F2, D1 D1 per team

* create a season-team dataframe
  
  ** number of wins/points/winning percentage

### estimate roster model 

- regress home win on the difference in number of home and visitor players by position and quality (predictor variables). Add a constant to the predictors and use OLS. The purpose is to deterimine the impact each roster positin has on home team success.

In [80]:
y = dm['HomeWin']  
X = sm.add_constant(dm[['DF1', 'DD1', 'DF2', 'DD2']] )

In [81]:
result = sm.OLS(y, X).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                HomeWin   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     2.508
Date:                Tue, 21 Nov 2017   Prob (F-statistic):             0.0819
Time:                        20:44:02   Log-Likelihood:                -733.26
No. Observations:                1015   AIC:                             1473.
Df Residuals:                    1012   BIC:                             1487.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.5147      0.016     32.281      0.000         0.483     0.546
DF1            0.0043      0.004      0.985      0.325        -0.004     0.013
DD1            0.0104      0.010      1.055      0.292        -0.009     0.030
DF2           -0.0043      0.004     -0.985      0.325        -0.013     0.004
DD2           -0.0104      0.010     -1.055      0.292        -0.030     0.009
==============================================================================
Omnibus:                        1.199   Durbin-Watson:                   1.890
Prob(Omnibus):                  0.549   Jarque-Bera (JB):              165.846
Skew:                          -0.084   Prob(JB):                     9.70e-37
Kurtosis:                       1.027   Cond. No.                     2.10e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.5e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [82]:
result.params

const    0.514692
DF1      0.004312
DD1      0.010381
DF2     -0.004312
DD2     -0.010381
dtype: float64

- By increasing the differential of **elite** player quality in forwards and defense (home team – visitor team) by one unit, home win **increases** by 0.4% and 1% respectfully.
- By increasing the differential of **secondary** player quality in forwards and defense (home team – visitor team) by one unit, home win **decreases** by 0.4% and 1% respectfully.